#cleaned-up TF-IDF approach

#keyword processing

In [1]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langdetect

In [3]:
import pandas as pd
import numpy as np

# keyword extraction and clean up
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

#Text Blob for sentiment analysis
from textblob import TextBlob

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import linear_kernel

import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

In [4]:
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"

In [5]:
hotel_df = pd.read_csv(url)

In [6]:
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive


In [7]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512470 entries, 0 to 512469
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Unnamed: 0                                  512470 non-null  int64  
 1   Hotel_Address                               512470 non-null  object 
 2   Additional_Number_of_Scoring                512470 non-null  int64  
 3   Review_Date                                 512470 non-null  object 
 4   Average_Score                               512470 non-null  float64
 5   Hotel_Name                                  512470 non-null  object 
 6   Reviewer_Nationality                        512470 non-null  object 
 7   Negative_Review                             512470 non-null  object 
 8   Review_Total_Negative_Word_Counts           512470 non-null  int64  
 9   Total_Number_of_Reviews                     512470 non-null  int64  
 

In [8]:
# combine reviews into one field
hotel_df["All_Reviews"] = hotel_df['Positive_Review'].astype(str) +"-"+ hotel_df["Negative_Review"]
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive,No real complaints the hotel was great great ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive,Location was good and staff were ok It is cut...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative,Great location in nice surroundings the bar a...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive,Amazing location and building Romantic settin...


In [9]:
#create random sample of original dataset - 5% sample
hotel_sample_df = hotel_df.sample(frac=.05)

In [10]:
hotel_sample_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
361055,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,13,2,10.0,"[' Leisure trip ', ' Family with young childre...",240 day,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...
270116,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,14,32,9.6,"[' Leisure trip ', ' Family with young childre...",136 day,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...
372022,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,2,2,5.0,"[' Leisure trip ', ' Couple ', ' Double Room '...",717 day,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...
136436,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,8,1,10.0,"[' Leisure trip ', ' Couple ', ' Superior Room...",305 day,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...
399377,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,13,2,10.0,"[' Leisure trip ', ' Group ', ' Comfort Room '...",610 day,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...


In [11]:
hotel_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25624 entries, 361055 to 443825
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  25624 non-null  int64  
 1   Hotel_Address                               25624 non-null  object 
 2   Additional_Number_of_Scoring                25624 non-null  int64  
 3   Review_Date                                 25624 non-null  object 
 4   Average_Score                               25624 non-null  float64
 5   Hotel_Name                                  25624 non-null  object 
 6   Reviewer_Nationality                        25624 non-null  object 
 7   Negative_Review                             25624 non-null  object 
 8   Review_Total_Negative_Word_Counts           25624 non-null  int64  
 9   Total_Number_of_Reviews                     25624 non-null  int64  
 10  Posi

In [12]:
hotel_sample_df = hotel_sample_df.reset_index(drop=True)
hotel_sample_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,13,2,10.0,"[' Leisure trip ', ' Family with young childre...",240 day,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...
1,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,14,32,9.6,"[' Leisure trip ', ' Family with young childre...",136 day,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...
2,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,2,2,5.0,"[' Leisure trip ', ' Couple ', ' Double Room '...",717 day,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...
3,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,8,1,10.0,"[' Leisure trip ', ' Couple ', ' Superior Room...",305 day,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...
4,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,13,2,10.0,"[' Leisure trip ', ' Group ', ' Comfort Room '...",610 day,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...


In [13]:
# Change data structure
#hotel_sample_df.drop(['Unnamed: 0', 'Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Positive_Review', 'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags', 'days_since_review'], axis=1) 

In [14]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from combined reviews
hotel_sample_pp_df = hotel_sample_df.copy()
key_words = []
name_key_words = []

r = Rake()

for index, row in hotel_sample_pp_df.iterrows():
    r.extract_keywords_from_text(row['All_Reviews'])
    key_words_dict_scores = r.get_word_degrees()
    keywords = list(key_words_dict_scores.keys())
    
    key_words.append(keywords)

    r.extract_keywords_from_text(row['Hotel_Name'])
    key_words_dict_scores = r.get_word_degrees()
    name_keywords = list(key_words_dict_scores.keys())
    
    name_key_words.append(name_keywords)
    
hotel_sample_pp_df["key_words"] = key_words
hotel_sample_pp_df["name_key_words"] = name_key_words

hotel_sample_pp_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words
0,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,10.0,"[' Leisure trip ', ' Family with young childre...",240 day,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...,"[comfortable, beds, important, friendly, helpf...","[hilton, london, wembley]"
1,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,9.6,"[' Leisure trip ', ' Family with young childre...",136 day,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...,"[breakfast, good, room, big, cozy, location, e...","[hotel, mailberger, hof]"
2,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,5.0,"[' Leisure trip ', ' Couple ', ' Double Room '...",717 day,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...,"[location, charged, 19, euro, mini, bar, use, ...","[expo, hotel, barcelona]"
3,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,10.0,"[' Leisure trip ', ' Couple ', ' Superior Room...",305 day,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...,"[large, comfortable, bed, nice, bedding, toile...","[mercure, paris, montmartre, sacr, coeur]"
4,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,10.0,"[' Leisure trip ', ' Group ', ' Comfort Room '...",610 day,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...,"[location, perfect, reminds, 5th, avenue, new,...","[royal, passeig, de, gracia]"


In [15]:
hotel_sample_pp_clean_df = hotel_sample_pp_df.copy()

In [16]:
# clean up keywords: drop symbols and numbers, as well as find root words

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


hotel_sample_pp_clean_df['clean_keywords']=hotel_sample_pp_clean_df['key_words'].map(lambda s:preprocess(s)) 
hotel_sample_pp_clean_df.head()


,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords
0,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,"[' Leisure trip ', ' Family with young childre...",240 day,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...,"[comfortable, beds, important, friendly, helpf...","[hilton, london, wembley]",comfortable beds important friendly helpful st...
1,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,"[' Leisure trip ', ' Family with young childre...",136 day,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...,"[breakfast, good, room, big, cozy, location, e...","[hotel, mailberger, hof]",breakfast good room big cozy location excellen...
2,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,"[' Leisure trip ', ' Couple ', ' Double Room '...",717 day,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...,"[location, charged, 19, euro, mini, bar, use, ...","[expo, hotel, barcelona]",location charged euro mini bar use repeated at...
3,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,"[' Leisure trip ', ' Couple ', ' Superior Room...",305 day,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...,"[large, comfortable, bed, nice, bedding, toile...","[mercure, paris, montmartre, sacr, coeur]",large comfortable bed nice bedding toilet made...
4,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,"[' Leisure trip ', ' Group ', ' Comfort Room '...",610 day,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...,"[location, perfect, reminds, 5th, avenue, new,...","[royal, passeig, de, gracia]",location perfect reminds avenue new york negative


#sentiment analysis

In [17]:
hotel_sub_df = hotel_sample_pp_clean_df.copy()

In [18]:
# run text blob in a loop

polarities = []
subjectivities = []

for index, row in hotel_sub_df.iterrows():
    sentence = row['All_Reviews']
    analysis = TextBlob(sentence).sentiment
    polarity = analysis[0]
    subjectivity = analysis[1]

    # populate lists
    polarities.append(polarity)
    subjectivities.append(subjectivity)


hotel_sub_df['sentiment_polarity'] = polarities
hotel_sub_df['sentiment_subjectivity']= subjectivities

hotel_sub_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...,"[comfortable, beds, important, friendly, helpf...","[hilton, london, wembley]",comfortable beds important friendly helpful st...,0.323750,0.700000
1,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...,"[breakfast, good, room, big, cozy, location, e...","[hotel, mailberger, hof]",breakfast good room big cozy location excellen...,0.225000,0.641667
2,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...,"[location, charged, 19, euro, mini, bar, use, ...","[expo, hotel, barcelona]",location charged euro mini bar use repeated at...,0.000000,0.000000
3,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...,"[large, comfortable, bed, nice, bedding, toile...","[mercure, paris, montmartre, sacr, coeur]",large comfortable bed nice bedding toilet made...,0.222279,0.549796
4,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...,"[location, perfect, reminds, 5th, avenue, new,...","[royal, passeig, de, gracia]",location perfect reminds avenue new york negative,0.278788,0.618182


In [29]:
hotel_sub_df.Hotel_Name.value_counts()

Britannia International Hotel Canary Wharf    247
Strand Palace Hotel                           220
Park Plaza Westminster Bridge London          212
Copthorne Tara Hotel London Kensington        180
Holiday Inn London Kensington                 164
                                             ... 
The Zetter Townhouse Clerkenwell                1
XO Hotel                                        1
Melia Paris Champs Elys es                      1
H tel Thoumieux                                 1
First Hotel Paris Tour Eiffel                   1
Name: Hotel_Name, Length: 1430, dtype: int64

#TF-IDF

In [19]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [20]:
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

#calculate cosine matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(hotel_sub_df['Hotel_Name'])

C:\Users\neil.ralston.MAC\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


[[1.         0.         0.         ... 0.03681013 0.         0.        ]
 [0.         1.         0.00654327 ... 0.03951106 0.         0.02159045]
 [0.         0.00654327 1.         ... 0.         0.         0.00730016]
 ...
 [0.03681013 0.03951106 0.         ... 1.         0.         0.08220099]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02159045 0.00730016 ... 0.08220099 0.         1.        ]]


In [21]:
#get recommendations based on tf-idf and cosine_similarities (hotel name)

def recommend_name(Hotel_Name, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == Hotel_Name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [30]:
recommend_name('Strand Palace Hotel')

,Hotel,Address,Rating,Score
0,The Park Grand London Paddington,1 3 Queens Garden Westminster Borough London W...,7.7,0.421848
1,Grange Blooms Hotel,7 Montague Street Bloomsbury Camden London WC1...,8.3,0.359205
2,The Park Grand London Paddington,1 3 Queens Garden Westminster Borough London W...,7.7,0.332887
3,Shangri La Hotel at The Shard London,31 St Thomas Street Southwark London SE1 9QU UK,9.4,0.310877
4,Crowne Plaza London Kensington,100 Cromwell Road Kensington and Chelsea Londo...,8.7,0.306609
5,Mercure London Bloomsbury Hotel,130 134 Southampton Row Camden London WC1B 5AF UK,8.3,0.297301
6,Arbor Hyde Park,36 37 Lancaster Gate Westminster Borough Londo...,8.4,0.297251
7,Hotel Am Stephansplatz,Stephansplatz 9 01 Innere Stadt 1010 Vienna Au...,9.3,0.287112
8,Kingsway Hall Hotel,66 Great Queen Street Covent Garden Camden Lon...,8.5,0.286939
9,London Marriott Hotel Marble Arch,134 George Street Westminster Borough London W...,8.2,0.284736


In [23]:
#get recommendations based on tf-idf and cosine_similarities (keyword)

def recommend_keyword(clean_keywords, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == clean_keywords].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec2_df = pd.DataFrame(data)
    
    return rec2_df

In [24]:
recommend_keyword('location')

IndexError: index 0 is out of bounds for axis 0 with size 0

#first review per hotel

In [25]:
hotel_unique_df = hotel_sub_df.drop_duplicates('Hotel_Name')

In [26]:
hotel_unique_df

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,362452,Lakeside Way Brent London HA9 0BU UK,1427,2016-12-06,8.8,Hilton London Wembley,United Kingdom,Nothing to dislike,5,4305,...,51.557696,-0.283526,UK,Positive,Comfortable beds very important Friendly help...,"[comfortable, beds, important, friendly, helpf...","[hilton, london, wembley]",comfortable beds important friendly helpful st...,0.323750,0.700000
1,270498,Annagasse 7 01 Innere Stadt 1010 Vienna Austria,142,2017-03-20,8.4,Hotel Mailberger Hof,Slovakia,The lift is a bit small Parking space is unde...,26,1781,...,48.204183,16.372666,Austria,Positive,The breakfast is good Room is big and cozy Lo...,"[breakfast, good, room, big, cozy, location, e...","[hotel, mailberger, hof]",breakfast good room big cozy location excellen...,0.225000,0.641667
2,373664,Mallorca 1 23 Sants Montju c 08014 Barcelona S...,390,2015-08-17,7.6,Expo Hotel Barcelona,Australia,We were charged 19 euro for the mini bar whic...,51,4646,...,41.379205,2.142799,Spain,Negative,Location- We were charged 19 euro for the min...,"[location, charged, 19, euro, mini, bar, use, ...","[expo, hotel, barcelona]",location charged euro mini bar use repeated at...,0.000000,0.000000
3,136528,3 Rue Caulaincourt 18th arr 75018 Paris France,159,2016-10-02,8.4,Mercure Paris Montmartre Sacr Coeur,United Kingdom,The toilet made you feel like you were sittin...,47,1652,...,48.885279,2.330102,France,Positive,Large comfortable bed with nice bedding - The...,"[large, comfortable, bed, nice, bedding, toile...","[mercure, paris, montmartre, sacr, coeur]",large comfortable bed nice bedding toilet made...,0.222279,0.549796
4,401068,Passeig de Gracia 84 Eixample 08008 Barcelona ...,393,2015-12-02,8.6,Royal Passeig de Gracia,Morocco,No Negative,0,3823,...,41.394678,2.162638,Spain,Positive,The location is perfect It reminds me about 5...,"[location, perfect, reminds, 5th, avenue, new,...","[royal, passeig, de, gracia]",location perfect reminds avenue new york negative,0.278788,0.618182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24277,111034,22 Rue Danielle Casanova 1st arr 75002 Paris F...,12,2016-03-20,8.8,Hotel Stendhal Place Vend me Paris MGallery by...,Saudi Arabia,Two items from the minibar were very expensive,9,141,...,48.868145,2.331306,France,Positive,A very comfortable hotel in a very nice area ...,"[comfortable, hotel, nice, area, arrived, earl...","[hotel, stendhal, place, vend, paris, mgallery...",comfortable hotel nice area arrived early morn...,0.195926,0.571296
24387,40505,12 rue des Saussaies 8th arr 75008 Paris France,29,2016-12-26,9.0,Le Pavillon des Lettres,United Arab Emirates,No Negative,0,164,...,48.871370,2.318119,France,Positive,what can I say the staff in this hotel is one...,"[say, staff, hotel, one, friendliest, ever, me...","[le, pavillon, des, lettres]",say staff hotel one friendliest ever met extre...,0.197917,0.541667
25061,18415,10 Bis rue du D barcad re 17th arr 75017 Paris...,26,2017-07-13,9.2,Hotel Le 10 BIS,United States of America,No Negative,0,196,...,48.878036,2.286491,France,Positive,Shower and airconditioning-No Negative,"[shower, airconditioning, negative]","[hotel, le, 10, bis]",shower airconditioning negative,-0.300000,0.400000
25465,287898,Boltzmanngasse 8 09 Alsergrund 1090 Vienna Aus...,61,2017-07-01,8.1,Hotel Boltzmann,Austria,Nothing,2,662,...,48.221678,16.356711,Austria,Positive,Fantastic breakfast Well worth the 10 upcharg...,"[fantastic, breakfast, well, worth, 10, upchar...","[hotel, boltzmann]",fantastic breakfast well worth upcharge air co...,0.566667,0.666667


In [27]:
hotel_unique_df2 = hotel_unique_df.drop(['Unnamed: 0', 'Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Review_Total_Positive_Word_Counts', 'Reviewer_Score', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags', 'lat', 'lng', 'days_since_review', 'Sentiment', 'All_Reviews'], axis=1) 

In [28]:
hotel_unique_df2.sort_values(by=['Average_Score'], ascending=False)

,Hotel_Address,Average_Score,Hotel_Name,Negative_Review,Positive_Review,Hotel_Country,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
4709,15 Place Vend me 1st arr 75001 Paris France,9.8,Ritz Paris,fine dining need Michelin star fine dining,Overall facility,France,"[overall, facility, fine, dining, need, michel...","[ritz, paris]",overall facility fine dining need michelin star,0.277778,0.333333
2914,Passeig de Colom 9 Ciutat Vella 08002 Barcelon...,9.6,Hotel The Serras,I would rate the hotel as very good four star...,Staff is very friendly and helpful Excellent ...,Spain,"[staff, friendly, helpful, excellent, location...","[hotel, serras]",staff friendly helpful excellent location goth...,0.224500,0.836000
623,Elisabets 11 Ciutat Vella 08001 Barcelona Spain,9.6,Hotel Casa Camper,There really wasn t a lot to not like in the ...,The staff were very friendly and helpful the ...,Spain,"[staff, friendly, helpful, breakfast, good, pr...","[hotel, casa, camper]",staff friendly helpful breakfast good provide ...,0.447917,0.505556
3679,1 Suffolk Place Westminster Borough London SW1...,9.6,Haymarket Hotel,The light levels in the room were not really ...,The staff the location and having a quiet room,UK,"[staff, location, quiet, room, light, levels, ...","[haymarket, hotel]",staff location quiet room light levels really ...,0.085185,0.533333
5048,Pau Claris 179 Eixample 08037 Barcelona Spain,9.6,H10 Casa Mimosa 4 Sup,The hotel is of an excellent standard in ever...,A stylish excellent quality hotel There is a ...,Spain,"[stylish, excellent, quality, hotel, beautiful...","[h10, casa, mimosa, 4, sup]",stylish excellent quality hotel beautiful welc...,0.306435,0.546961
...,...,...,...,...,...,...,...,...,...,...,...
9414,346 348 Edgware Road Westminster Borough Londo...,6.6,Best Western Maitrise Hotel Edgware Road,Hard to find hotel reception is behind the ba...,The shower was nice That s about it,UK,"[shower, nice, hard, find, hotel, reception, b...","[best, western, maitrise, hotel, edgware, road]",shower nice hard find hotel reception behind b...,-0.129762,0.751190
2435,28 Ebury Street Westminster Borough London SW1...,6.6,The Tophams Hotel,Have to say that the water pressure for the s...,Location near Victoria was great for a quick ...,UK,"[location, near, victoria, great, quick, overn...","[tophams, hotel]",location near victoria great quick overnighter...,0.089167,0.630000
2490,Ferdinand Bolstraat 194 Oud Zuid 1072 LW Amste...,6.4,Savoy Hotel Amsterdam,The room was too small for the money paid,No Positive,Netherlands,"[positive, room, small, money, paid]","[savoy, hotel, amsterdam]",positive room small money paid,-0.181818,0.472727
107,75 Gower Street Camden London WC1E 6HJ UK,6.4,Hotel Cavendish,The fact that I cannot to this day collect th...,The location,UK,"[location, fact, cannot, day, collect, invoice...","[hotel, cavendish]",location fact cannot day collect invoice month...,0.000000,0.750000


In [ ]:
#add in minimum rating filter either js or python, last cell .loc where rating greater than 9